In [13]:
import os
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
from torchvision import transforms

import os
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd
from torchvision.models import ResNet50_Weights

In [14]:
def resize(img, target_size=256):
    w, h = img.size
    ratio = target_size / h if w > h else target_size / w
    return img.resize((int(w*ratio), int(h*ratio)))


hyperparams = {
    "train_val_ratio" : 0.9,
    "batch_size" : 350,
    "learning_rate" : 0.0001,
    "epochs" : 3,
    "transform" : transforms.Compose(
        [
            transforms.Lambda(resize),
            # transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48235, 0.45882, 0.40784],
                std=[1.0/255.0, 1.0/255.0, 1.0/255.0]
            )
        ]
    )
}

dataset = ImageFolder("./data/train", transform=hyperparams["transform"])

# Train, Val 데이터 분리
train_size = int(len(dataset) * hyperparams["train_val_ratio"])
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Train, Val 데이터 로더 정의
train_dataloader = DataLoader(
    train_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True
)
val_dataloader = DataLoader(
    val_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True
)

In [15]:
import os
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd


device = "mps" if torch.mps.is_available() else "cpu"
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, len(dataset.classes))
model.eval().to(device)

transform = transforms.Compose([
    transforms.Lambda(resize),
    # transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.48235, 0.45882, 0.40784],
        std=[1.0/255.0, 1.0/255.0, 1.0/255.0]
    )
])

test_dir = "./data/test"
image_files = sorted([f for f in os.listdir(test_dir) if f.endswith(".jpg")])

results = []

for fname in image_files:
    path = os.path.join(test_dir, fname)
    img = Image.open(path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred = torch.argmax(output, dim=1).item()
        pred_label = dataset.classes[pred]


    image_id = os.path.splitext(fname)[0]
    results.append((image_id, pred_label))

df = pd.DataFrame(results, columns=["ID", "rock_type"])


sub_path = "./rock_submission.csv" 
if os.path.exists(sub_path):
    os.remove(sub_path)
df.to_csv(sub_path, index=False)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/simon/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 60.8MB/s]
